In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import pickle

In [3]:
#load the trained model
model = load_model('model.h5')
#load the scaler
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

#load the encorder
with open('onehot_encoder_geo.pkl', 'rb') as f:
    label_encoder_geo = pickle.load(f)
#load the label encoder
with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender = pickle.load(f)


In [22]:
#Example input data

input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}


In [23]:
#one hot encode the categorical features
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]])
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

d:\annclassification\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [24]:
# Convert input data to DataFrame
input_df = pd.DataFrame([input_data])
input_df 

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [25]:
#Encode the categorical features
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [26]:
# Concatenate the one-hot encoded geography with the input DataFrame
#input_df = pd.concat([input_df, geo_encoded_df], axis=1)
# Drop the original categorical columns
#input_df.drop(['Geography'], axis=1, inplace=True)

input_df = pd.concat([input_df.drop(['Geography'], axis=1), geo_encoded_df], axis=1)

input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [27]:
#Scale the input data
input_df_scaled = scaler.transform(input_df)

In [29]:
#predict churn
predictions = model.predict(input_df_scaled)

prediction_probability = predictions[0][0]
# Print the prediction probability
print("Prediction Probability:", prediction_probability)
if prediction_probability > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is unlikely to churn.")


# Convert predictions to binary (0 or 1)
#predictions_binary = (predictions > 0.5).astype(int)
# Print the predictions
#print("Predicted Churn Probability:", predictions[0][0])
#print("Predicted Churn (0 = No, 1 = Yes):", predictions_binary[0][0])

1/1 [==============================] - 0s 70ms/step
Prediction Probability: 0.025807513
The customer is unlikely to churn.
